## Exercise to introduce cross-correlations of signals from a single source recorded at different stations

## Main Python packages used: 

- [**NumPy**](https://numpy.org) : for mathematical functions

- [**Matplotlib**](https://matplotlib.org) : for plotting results

- [**SciPy**](https://github.com/obspy/obspy/wiki) : to compute [**cross-correlations**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.correlate.html)

---


### Part 1: cross-correlation of simple impulsive signals

#### Synthetic impulsive source radiates a single Ricker wavelet

#### Random noise is simulated as sum of many Ricker wavelets with random amplitudes and times

---

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

#==============================================================
#------------------------------------------------
# parameters to modify
#------------------------------------------------

azimuth = 90            # azimuth toward the station
siglevel = 1.            # signal level
noiselevel = 0.       # noise level

#==============================================================



#------------------ plotting mode
%matplotlib widget
#----------------------------


plt.close('all')


#------------------------------------------------
# subroutines for generating impulsive sgnal and noise
#------------------------------------------------

#-------------------------- single Ricker pulse
def one_ricker(t,delay,f):
    a = np.pi*np.pi*f*f*(t-delay)**2
    return (1-2*a)*np.exp(-a)


#-------------------------- random signal as sum of many Ricker pulses
def random_rickers(t,nsig,delay,f,lam):
    a = np.random.rand(nsig)
    r = np.random.rand(nsig)
    dts = -np.log(r)/lam
    ts = np.cumsum(dts)-250
    u = 0*t
    for i in range (0, nsig):
        u = u + a[i]*one_ricker(t,ts[i]+delay,f)
    return u
        

#------------------------------------------------
# defining parmeters

dist = 500
vel = 5

npt = 700
nsig = 2000
lam = .3


dt = 0.5
f = .07             # ricker frequency

#------------------------------------------------
angr = np.pi*azimuth/180
delay1 = 150;
delay2 = delay1 + dist*np.sin(angr)/vel

#------------------------------------------------
# defining variables
#..............................

#----------------- signal time
t = np.arange(npt)
t = dt*t

#----------------- correlation time
tcc = np.arange(-(npt-1),npt)
tcc = dt*tcc

#------------------------------------------------
# computing signal
#..............................

sig1 = siglevel*one_ricker(t,delay1,f)
sig2 = siglevel*one_ricker(t,delay2,f)


#------------------------------------------------
# adding noise
#..............................

sig1 = sig1 + noiselevel*random_rickers(t,nsig,0,f,lam)
sig2 = sig2 + noiselevel*random_rickers(t,nsig,0,f,lam)


#------------------------------------------------
# computing cross-correlation
#..............................

cc = signal.correlate(sig2,sig1,'full')/np.sqrt(np.sum(sig2**2)*np.sum(sig1**2))


#------------------------------------------------
# plotting source-station geometry
#..............................
plt.figure()

plt.axes().set_aspect('equal')


yr1 = 1250
xr1 = 1500
plt.plot(xr1,yr1,'r^')

yr2 = 1250
xr2 = 1000
plt.plot(xr2,yr2,'b^')

xs = 750*np.sin(angr)+1250
ys = 750*np.cos(angr)+1250
plt.plot(xs,ys,'k*')
    
plt.ylim(0,2500)
plt.xlim(0,2500)

plt.axhline(y=1250,color='k',linestyle=':',linewidth=1)
plt.axvline(x=1250,color='k',linestyle=':',linewidth=1)

plt.title('source (star) and receivers (triangle) positions')
plt.xlabel('distance (km)')
plt.ylabel('distance (km)')


plt.show()





#------------------------------------------------
# plotting signals
#..............................

plt.figure()

ax1 = plt.subplot(3,1,1)
ax1.plot(t,sig1,'r')
plt.title('signal at receiver 1')
plt.xlim(0,300)

ax2 = plt.subplot(3,1,2)
ax2.plot(t,sig2,'b')
plt.title('signal at receiver 2')
plt.xlim(0,300)

ax3 = plt.subplot(3,1,3)
ax3.plot(tcc,cc,'k')
plt.xlim(-150,150)
plt.title('cross-correlations between receivers 1 and 2')
plt.xlabel('time (s)')

plt.subplots_adjust(top=0.92, bottom=0.12, left=0.10, right=0.95, hspace=0.55, wspace=0.35)

plt.show()


---

### Part 2: cross-correlation of continuous and random signals

---

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

#==============================================================
#------------------------------------------------
# parameters to modify
#------------------------------------------------

azimuth = 90            # azimuth toward the station
siglevel = 1.            # signal level
noiselevel = 1.       # noise level

#==============================================================





#------------------ plotting mode
%matplotlib widget
#----------------------------


plt.close('all')



#------------------------------------------------
# subroutines for generating impulsive sgnal and noise
#------------------------------------------------

#-------------------------- single Ricker pulse
def one_ricker(t,delay,f):
    a = np.pi*np.pi*f*f*(t-delay)**2
    return (1-2*a)*np.exp(-a)


#-------------------------- random signal as sum of many Ricker pulses
dt = 0.5
f = .07             # ricker frequency
n_rick = 25

trick = np.arange(-n_rick,n_rick)*dt
rick0 = one_ricker(trick,0,f)

def fast_random_rickers(t,nsig,delay,f,lam,nt,dt):
    a = 2*np.random.rand(nsig)-1
    r = np.random.rand(nsig)
    dts = -np.log(r)/lam
    ts = np.cumsum(dts)-250
    tmax = nt*dt
    drange = 1/f
    u = 0*t
    for i in range (0, nsig):
        delI = ts[i]+delay
        if delI > tmax:
            break
        dif_t = np.around((t-delI)/dt)
        i_dif_t = dif_t.astype(int)
        idx = (i_dif_t>-n_rick) & (i_dif_t<n_rick)
        ii = np.around((t[idx]-delI)/dt)
        iii = ii.astype(int)+n_rick
        u[idx] = u[idx] + a[i]*rick0[iii]
    return u





#------------------------------------------------
# defining parmeters

dist = 500
vel = 5

npt = 5000
nsig = 8000
lam = .3


#dt = 0.5
#f = .07             # ricker frequency

#------------------------------------------------
angr = np.pi*azimuth/180
delay1 = 150;
delay2 = delay1 + dist*np.sin(angr)/vel


#------------------------------------------------
# defining variables
#..............................

#------------------ signal time
t = np.arange(npt)
t = dt*t

#------------------ corrleations time
tcc = np.arange(-(npt-1),npt)
tcc = dt*tcc


#------------------------------------------------
# computing coherent signal
#..............................
seed = np.random.randint(100000,size=1)
np.random.seed(seed)
sig1 = fast_random_rickers(t,nsig,delay1,f,lam,npt,dt)
np.random.seed(seed)
sig2 = fast_random_rickers(t,nsig,delay2,f,lam,npt,dt)


#------------------------------------------------
# adding incoherent noise
#..............................
seed = np.random.randint(100000,size=1)
np.random.seed(seed)
sig1 = siglevel*sig1 + noiselevel*fast_random_rickers(t,nsig,delay1,f,lam,npt,dt)
sig2 = siglevel*sig2 + noiselevel*fast_random_rickers(t,nsig,delay1,f,lam,npt,dt)


#------------------------------------------------
# computing cross-correlation
#..............................

cc = signal.correlate(sig2,sig1,'full')/np.sqrt(np.sum(sig2**2)*np.sum(sig1**2))


#------------------------------------------------
# plotting source-station geometry
#..............................
plt.figure()

plt.axes().set_aspect('equal')


yr1 = 1250
xr1 = 1500
plt.plot(xr1,yr1,'r^')

yr2 = 1250
xr2 = 1000
plt.plot(xr2,yr2,'b^')

xs = 750*np.sin(angr)+1250
ys = 750*np.cos(angr)+1250
plt.plot(xs,ys,'k*')

    
plt.ylim(0,2500)
plt.xlim(0,2500)

plt.axhline(y=1250,color='k',linestyle=':',linewidth=1)
plt.axvline(x=1250,color='k',linestyle=':',linewidth=1)

plt.title('source (star) and receivers (triangle) positions')
plt.xlabel('distance (km)')
plt.ylabel('distance (km)')


plt.show()


#------------------------------------------------
# plotting signals
#..............................

plt.figure()

plt.subplot(3,1,1)
plt.plot(t,sig1,'r')
plt.title('signal at receiver 1')

plt.subplot(3,1,2)
plt.plot(t,sig2,'b')
plt.title('signal at receiver 2')

plt.subplot(3,1,3)
plt.plot(tcc,cc,'k')
plt.xlim(-300,300)
plt.title('cross-correlations between receivers 1 and 2')
plt.xlabel('time (s)')

plt.subplots_adjust(top=0.92, bottom=0.12, left=0.10, right=0.95, hspace=0.55, wspace=0.35)

plt.show()
